In [ ]:
import cv2
import os
import numpy as np
import random
from skimage.feature import local_binary_pattern
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_input(x):
    x = x.astype('float32')
    x /= 127.5
    x -= 1.
    return x

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/Train_rash/Train_2_Cases"
val_path = "/Users/niteshyadav/Lyme_Disease/Validation_rash/Validation_2_Cases"

def load_image_paths_and_labels(data_dir): 
    image_paths = []
    labels = []
    for label in ['Positive', 'Negative']:
        label_dir = os.path.join(data_dir, label)
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(1 if label == 'Positive' else 0)  # Convert labels to numerical values
    return image_paths, labels

# Load paths and labels for training and validation data
train_image_paths, train_labels = load_image_paths_and_labels(train_path)
val_image_paths, val_labels = load_image_paths_and_labels(val_path)

def initialize_population(image, pop_size=10):
    population = []
    
    if image is None:
        raise ValueError("Image is empty. Check the file path.")
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Thresholding to find high-intensity regions
    _, thresh = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    
    # Red color mask
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    red_mask = mask1 + mask2
    
    # Combine masks
    combined_mask = cv2.bitwise_and(thresh, red_mask)
    
    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize population with bounding boxes of detected regions
    for contour in contours[:pop_size]:
        x, y, w, h = cv2.boundingRect(contour)
        population.append((x, y, w, h))
    
    # If not enough regions, fill with random regions
    while len(population) < pop_size:
        h, w = gray_image.shape
        x = random.randint(0, w-1)
        y = random.randint(0, h-1)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        population.append((x, y, width, height))
    
    return population

def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Calculate redness
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_roi, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_roi, lower_red, upper_red)
    red_mask = mask1 + mask2
    red_area = np.sum(red_mask) / (w * h)
    
    # Shape: prefer circular or oval shapes
    aspect_ratio = float(w) / h
    shape_score = 1.0 if 0.75 <= aspect_ratio <= 1.25 else 0.5
    
    # Texture: use LBP to score texture
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    texture_score = np.mean(hist)
    
    # Combine scores
    fitness = red_area * 0.4 + shape_score * 0.3 + texture_score * 0.3
    return fitness

def crossover(parent1, parent2):
    x1, y1, w1, h1 = parent1
    x2, y2, w2, h2 = parent2
    child1 = (x1, y2, w1, h2)
    child2 = (x2, y1, w2, h1)
    return child1, child2

def mutate(candidate, image_shape, mutation_rate=0.1):
    if random.random() < mutation_rate:
        h, w, _ = image_shape
        x, y, w, h = candidate
        x = random.randint(0, w-1)
        y = random.randint(0, h-1)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        candidate = (x, y, width, height)
    return candidate

def select(population, fitnesses, num_to_select):
    selected = list(zip(population, fitnesses))
    selected.sort(key=lambda x: x[1], reverse=True)
    return [candidate for candidate, fitness in selected[:num_to_select]]

def genetic_algorithm(image, num_generations=20, pop_size=10, mutation_rate=0.1):
    population = initialize_population(image, pop_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(candidate, image) for candidate in population]
        population = select(population, fitnesses, pop_size // 2)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, image.shape, mutation_rate))
            new_population.append(mutate(child2, image.shape, mutation_rate))
        population = new_population
    best_candidate = max(population, key=lambda candidate: fitness_function(candidate, image))
    return best_candidate

def extract_roi_and_preprocess(image_paths, labels):
    preprocessed_images = []
    filtered_labels = []
    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        
        # Check if the image is loaded successfully
        if image is None:
            print(f"Warning: Could not load image {image_path}. Skipping.")
            continue
        
        detected_rash = genetic_algorithm(image)
        x, y, w, h = detected_rash
        roi = image[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (300, 300))
        preprocessed_images.append(roi_resized)
        filtered_labels.append(label)  # Keep the label in sync
    
    return np.array(preprocessed_images), np.array(filtered_labels)

# Preprocess the ROIs
X_train, train_labels_filtered = extract_roi_and_preprocess(train_image_paths, train_labels)
X_val, val_labels_filtered = extract_roi_and_preprocess(val_image_paths, val_labels)

# Data augmentation for training
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   zoom_range=0.1)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(X_train, train_labels_filtered, batch_size=8)
val_generator = val_datagen.flow(X_val, val_labels_filtered, batch_size=8)

# Define the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_generator, epochs=100, validation_data=val_generator, shuffle=True)

# Save the model weights
model.save_weights("model_with_roi_extraction.h5")


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


Epoch 1/100


/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 221s 5s/step - accuracy: 0.5048 - loss: 2.6010 - val_accuracy: 0.5862 - val_loss: 0.6770
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 223s 5s/step - accuracy: 0.5065 - loss: 2.4284 - val_accuracy: 0.5747 - val_loss: 0.6877
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 221s 5s/step - accuracy: 0.4849 - loss: 2.1853 - val_accuracy: 0.5862 - val_loss: 0.6804
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 215s 5s/step - accuracy: 0.5243 - loss: 2.6200 - val_accuracy: 0.5862 - val_loss: 0.7338
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 208s 5s/step - accuracy: 0.4687 - loss: 2.5496 - val_accuracy: 0.5862 - val_loss: 0.6978
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 213s 5s/step - accuracy: 0.5720 - loss: 1.9316 - val_accuracy: 0.5862 - val_loss: 0.7844
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 213s 5s/step - accuracy: 0.4848 - loss: 2.3962 - val_accuracy: 0.5632 - val_loss: 0.6855
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 230s 5s/step - accuracy: 0.5338 - loss: 2.0095 - val_accuracy: 0.5862 - val_l